In [ ]:
pip install opencv-contrib-python==3.4.1.15

     |████████████████████████████████| 30.3MB 113kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
import cv2 as cv
import numpy
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
import os
from google.colab import drive
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
#I store the image on Google drives so if you wannt to run this cell, please upload data on Google Drive. Sorry for this inconvenience!
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get The corner key points by Harris and represent by SIFT


In [ ]:
def get_feature():
  list_feature = []
  extractor = cv.xfeatures2d.SIFT_create()
  harris_laplace	=	cv.xfeatures2d.HarrisLaplaceFeatureDetector_create(maxCorners=1000)
  i = 0
  for image_path in os.listdir('drive/My Drive/images'):
    image = cv.imread('drive/My Drive/images/'+image_path)
    # plt.imshow(image)
    gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    kp = harris_laplace.detect(gray,None)
    keypoint, desc = extractor.compute(gray,kp)
    list_feature.append(desc)
    # i = i+1
    # if i == 100:
    #   break;
  return list_feature

In [ ]:
feature_list = get_feature()

In [ ]:
list_feature = []
for i in feature_list:
  list_feature.extend(i)

# Clustering

In [ ]:
number_clusters = 1000
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(list_feature)

# Calcualte TF-IDF

In [ ]:
hist_images = []
h = 0
for i in feature_list:
  tempt = [0]*number_clusters
  for j in range(0,len(i)):
    tempt[kmeans.labels_[h+j]] = tempt[kmeans.labels_[h+j]] +1
  h = h + len(i)
  hist_images.append(tempt)

In [ ]:
idf = [0]* number_clusters
for i in hist_images:
  for j in range(0,len(i)):
    if i[j] != 0:
      idf[j] = idf[j] + 1

In [ ]:
image_number = len(feature_list)
for i in hist_images:
  for j in range (0,len(i)):
    if i[j] != 0:
      i[j] = i[j] * (image_number/idf[j]) 

# Search Image


In [ ]:
extractor = cv.xfeatures2d.SIFT_create()
harris_laplace	=	cv.xfeatures2d.HarrisLaplaceFeatureDetector_create(maxCorners=1000)
image = cv.imread('drive/My Drive/images/bodleian_000264.jpg')
# cv2_imshow(image)
gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
kp = harris_laplace.detect(gray,None)
keypoint, desc = extractor.compute(gray,kp)
predict_cluster = kmeans.predict(desc,None)
hist = [0] *number_clusters
for i in predict_cluster:
  hist[i] = hist[i] + 1
for i in  range(0,len(hist)):
  if hist[i] != 0:
    hist[i] = hist[i] * (image_number/idf[i]) 

In [ ]:
vec1 = np.array([hist])
vec2 = np.array(hist_images)
result = cosine_similarity(vec1, vec2)
dict_image = {}
for i,j in zip(result[0],os.listdir('drive/My Drive/images')):
  dict_image[i] = j

In [ ]:
result[0] = np.sort(result[0])
result[0] = result[0][::-1]
for i in range (0,10):
  print(dict_image[result[0][i]],result[0][i])
  image = cv.imread('drive/My Drive/images/'+dict_image[result[0][i]])
  cv2_imshow(image)